In [1]:
import pandas
import math
import json
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

page_to_class = {}
page_to_class['a'] = 'H'
page_to_class['b'] = 'AI'
page_to_class['c'] = 'AI'
page_to_class['d'] = 'AI'


page_to_algo = {}
page_to_algo['a'] = 'humans'
page_to_algo['b'] = 'azure'
page_to_algo['c'] = 'mccq'
page_to_algo['d'] = 'google'

jsonpath = './Data_all/'
contents = pandas.read_csv(jsonpath + 'out.csv')

count = 0
trials_dict = {}
workercount = 0
results = {'mturk':[]}
key = 'mturk'

person_to_personal_df = {}   
for row in contents.iterrows():
    row_data = row[1]['datastring']
    if isinstance(row_data,str):
        
        data_dict = json.loads(row_data)
        workercount += 1
        
#         person_age = person_questionnaire['age']
#         person_gender = person_questionnaire['gender']
#         person_education = person_questionnaire['education']
#         person_df = pandas.DataFrame(columns = ['image','topic_match','response_class','actual_class','class_match','response_gender'])
        #for i in range(1,26):
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
            
                count += 1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                #print(data_dict['workerId'])
                trial_data['workerData'] = data_dict['questiondata']
                curres = dat['trialdata']

                rt = curres['rt'] ###
                hit = curres['hit'] ###
                trial = curres['trial']
                
                imageID = curres['imageID']
                topic_match = int(curres['topic']) == int(curres['oneWord_curr']) + 1
                
                if not topic_match:
                    continue

                response_class = curres['Aclass']
                groundtruth = page_to_class[curres['page_name'].split('_')[1]]
                algo = page_to_algo[curres['page_name'].split('_')[1]]
                page_name = curres['page_name']
                
                response_speaker = response_class ###
                if response_speaker == 'AI':
                    response_speaker = 'machine'
                    
                if response_speaker == 'H':
                    response_speaker = 'human'
                    
                    
                if  groundtruth == 'H':
                    machine_groundtruth = ''
                    human_groundtruth = 'human'
                    groundtruth = 'human'
                else:
                    groundtruth = 'machine'
                    real_gender = ''
                    machine_groundtruth = algo
                    human_groundtruth = ''
                
                guess_gender = curres['Agender']
                if guess_gender == 'F': 
                    guess_gender_long = 'female' # guess gender
                else:
                    guess_gender_long = 'male' # guess gender

                counterbalance = 0
                trialData_save = {'rt':rt, 'hit':hit, 'trial':trial, 
                              'response_speaker':response_speaker,
                              'machine_groundtruth':machine_groundtruth,'imageID':imageID,
                              'page_name': page_name, 'counterbalance':counterbalance,
                              'groundtruth':groundtruth,
                              'human_groundtruth':human_groundtruth ,
                              'response_gender':guess_gender_long}
                
                trial_data['trialData'] = trialData_save
                trials_dict['Trial_'+str(count)] = trial_data
                
results[key] = trials_dict    

print('All data is extracted.')
for key in results:
    print('There are [' + key + '; ' + str(len(results[key])) + ']')
    
#save the compiled results as jason
with open("./savedResults/compiled.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

All data is extracted.
There are [mturk; 1416]
Compilation completed! Results are saved to folder!
